# Re-grid mirrored CMIP6 data on ACDN

Use this notebook to re-grid all of the mirrored CMIP6 data to a common grid. We will use the grid that is common between CESM2, CESM2-WACCM, NorESM2, and TaiESM1.

Instead of just re-gridding all files, we will take a manifest specific approach, and re-grid by model, frequency, variable, etc. I think these regridded files should be saved in their own folder, since it is sort of a derived product? I.e., "CMIP6 outputs on a common grid" feels like its own thing. The alternative would be to save regridded files in this native ESGF directory structure, but I worry that could cause confusion because then we have derived data mixed in with mirrored data. It seems like it might be nice if the `/beegfs/CMIP6/arctic-cmip6/CMIP6` directory excluded everything but mirrored data. So, we will make another directory for the regridded data: `/beegfs/CMIP6/arctic-cmip6/CMIP6_common`

Test re-gridding historical monthly data from one grid to another

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr
import xesmf as xe

from config import *

# ignore serializationWarnings from xarray for datasets with multiple FillValues
import warnings
warnings.filterwarnings("ignore", category=xr.SerializationWarning)

In [6]:
regrid_batch_dir.joinpath(batch_tmp_fn.format(model="CESM2", scenario="historical"))

PosixPath('/center1/SNAP/kmredilla/cmip6_regridding/regrid_batch/batch_CESM2_historical.txt')

Iterate over the batch files and create slurm `sbatch` scripts to regrid all filepaths contained in those files (one job for each file).

In [ ]:
job_ids = [slurm.submit_sbatch(fp) for fp in sbatch_fps]

In [ ]:
# target grid: /beegfs/CMIP6/arctic-cmip6/CMIP6/ScenarioMIP/NCAR/CESM2/ssp370/r11i1p1f1/Amon/tas/gn/v20200528/tas_Amon_CESM2_ssp370_r11i1p1f1_gn_206501-210012.nc